# Project 2 NLP: Hatespeech Classifier

## Authors:

Adrian Obermühlner & Freja Rasmussen

## Resarch Question:

How do different preprocessing methods (nothing, stop word removal, lemming, stemming,…) affect the result of a hate speech classifier?

## Imports

In [98]:
# Imports
import pandas as pd
import numpy as np
import torch
import regex as re
import matplotlib.pyplot as plt

# Preprocessing imports
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Tokenizing
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [99]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1650 Ti


## Data Import


In [100]:
RANDOM_SEED = 42
BINARY_LABEL = "is_hate"
CATEGORIES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

np.random.seed(RANDOM_SEED)  # set random seed for reproducibility
# Make the labels into hate and no hate as 1 and 0

def binarize_labels(df):
    return (df[CATEGORIES].sum(axis=1) > 0).astype(int)

data_train = pd.read_csv("./data/train/train.csv", index_col=0)
data_train[BINARY_LABEL] = binarize_labels(data_train)

data_test = pd.read_csv("./data/test/test.csv", index_col=0).join(
    pd.read_csv("./data/test_labels/test_labels.csv", index_col=0)
)
data_test.drop(data_test[data_test["toxic"] == -1].index, inplace=True)
data_test[BINARY_LABEL] = binarize_labels(data_test)

In [101]:
data_train['comment_text'].head(10)

id
0000997932d777bf    Explanation\nWhy the edits made under my usern...
000103f0d9cfb60f    D'aww! He matches this background colour I'm s...
000113f07ec002fd    Hey man, I'm really not trying to edit war. It...
0001b41b1c6bb37e    "\nMore\nI can't make any real suggestions on ...
0001d958c54c6e35    You, sir, are my hero. Any chance you remember...
00025465d4725e87    "\n\nCongratulations from me as well, use the ...
0002bcb3da6cb337         COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
00031b1e95af7921    Your vandalism to the Matt Shirvington article...
00037261f536c51d    Sorry if the word 'nonsense' was offensive to ...
00040093b2687caa    alignment on this subject and which are contra...
Name: comment_text, dtype: object

In [102]:
data_test.head(10)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_hate
id,,,,,,,,
0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,0
000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,0
0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,0
0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,0
00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,0
000663aff0fffc80,this other one from 1897,0,0,0,0,0,0,0
000689dd34e20979,== Reason for banning throwing == \n\n This ar...,0,0,0,0,0,0,0
000844b52dee5f3f,|blocked]] from editing Wikipedia. |,0,0,0,0,0,0,0
00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but ...",1,0,0,0,0,0,1


In [103]:
# get the distribution of the labels to see if roughly similar for both

is_hate_count_train = data_train['is_hate'].value_counts()
ratio_train = is_hate_count_train/ len(data_train)

is_hate_count_test = data_test['is_hate'].value_counts()
ratio_test = is_hate_count_test/ len(data_test)

print('Ratio of no/is hate for train set: ', ratio_train)
print('Ratio of no/is hate for test set: ', ratio_test)

Ratio of no/is hate for train set:  0    0.898321
1    0.101679
Name: is_hate, dtype: float64
Ratio of no/is hate for test set:  0    0.90242
1    0.09758
Name: is_hate, dtype: float64


## Representation

## Data Preprocessing

**Note**: We would need to make a loop for the different combinations of 
preprocessing (none, only stemming, only lemming, only stop word removal and every combination of this)
Either as coloumns that can be used to iterate over for the model training and validation, or make the preprocessing
and then go further and repeat from beginning.


In [104]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def handle_negations(tokens):
    negation_words = {'not', "n't", 'no', 'never', 'none'}
    processed_tokens = []
    skip_next = False

    for i, word in enumerate(tokens):
        if skip_next:
            skip_next = False
            continue

        if word in negation_words and i + 1 < len(tokens):
            processed_tokens.append(word + '_' + tokens[i + 1])
            skip_next = True
        else:
            processed_tokens.append(word)

    return processed_tokens

import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


# Assuming stop_words is defined somewhere

def get_wordnet_pos(treebank_tag):
    """Converts treebank tags to wordnet tags."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


def preprocess_text(text, use_lower=False, remove_stopwords=False, use_stemming=False, use_lemming=False, combine_negations=False, keep_semantic_punctuation=False):
    if use_lower:
        text = text.lower()
    
    tokens = word_tokenize(text)
    
    if combine_negations:
        tokens = handle_negations(tokens)

    if remove_stopwords:
        tokens = [word for word in tokens if word.lower() not in stop_words]
    
    if not keep_semantic_punctuation:
        tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens]
    
    if use_stemming:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
    elif use_lemming:
        lemmatizer = WordNetLemmatizer()
        tagged_tokens = pos_tag(tokens)
        tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]
    
    filtered_tokens = [word for word in tokens if word.isalpha()]

    return ' '.join(filtered_tokens)


# Apply the preprocessing function to the training and test datasets
# We don't pass the rare_words parameter, so rare word removal is not performed
#data_train['comment_text_clean_2'] = data_train['comment_text'].apply(lambda x: preprocess_text(x, use_stemming=False))
#data_test['comment_text_clean_2'] = data_test['comment_text'].apply(lambda x: preprocess_text(x, use_stemming=False))



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flras\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flras\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flras\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\flras\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [105]:
data_train.shape

(159571, 8)

In [106]:
# Keeping semantic punctuation (keeping ! and ?)
data_train['text_no_punctuation'] = data_train['comment_text'].apply(lambda x: preprocess_text(x, use_lower=True, remove_stopwords=True, keep_semantic_punctuation=False))
data_train['text_with_punctuation'] = data_train['text_no_punctuation'].apply(lambda x: preprocess_text(x, keep_semantic_punctuation=True,))


In [107]:
# Keeping semantic punctuation (keeping ! and ?)
data_test['text_no_punctuation'] = data_test['comment_text'].apply(lambda x: preprocess_text(x, use_lower=True, remove_stopwords=True, keep_semantic_punctuation=False))
data_test['text_with_punctuation'] = data_test['text_no_punctuation'].apply(lambda x: preprocess_text(x, keep_semantic_punctuation=True,))


In [108]:
data_train.to_csv('train_all_coloumns.csv')

In [109]:
data_test.to_csv('test_all_coloumns.csv')

In [110]:
# Keep negations
data_train['text_without_negations'] = data_train['comment_text'].apply(lambda x: preprocess_text(x, use_lower=True, remove_stopwords=True, keep_semantic_punctuation=False, use_lemming=True, combine_negations=False))
data_train['text_with_negations'] = data_train['text_without_negations'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=False, use_lemming=False, combine_negations=True))

data_train.to_csv('train_all_coloumns.csv')

In [111]:
# Keep negations
data_test['text_without_negations'] = data_test['comment_text'].apply(lambda x: preprocess_text(x, use_lower=True, remove_stopwords=True, keep_semantic_punctuation=False, use_lemming=True, combine_negations=False))
data_test['text_with_negations'] = data_test['text_without_negations'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=False, use_lemming=False, combine_negations=True))

data_test.to_csv('test_all_coloumns.csv')

In [112]:
# 5 Steps of Processing
# Only lowercase
data_train['data_text_1'] = data_train['comment_text'].apply(lambda x: preprocess_text(x, use_lower=True, remove_stopwords=False, keep_semantic_punctuation=True, use_stemming=False))
data_train.to_csv('train_all_coloumns.csv')

In [113]:
# 5 Steps of Processing
# Only lowercase
data_test['data_text_1'] = data_test['comment_text'].apply(lambda x: preprocess_text(x, use_lower=True, remove_stopwords=False, keep_semantic_punctuation=True, use_stemming=False))
data_test.to_csv('test_all_coloumns.csv')

In [114]:

# Lowercase and stopwords removal
data_train['data_text_2'] = data_train['data_text_1'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=True, keep_semantic_punctuation=False, use_stemming=False))
data_train.to_csv('train_all_coloumns.csv')

In [115]:

# Lowercase and stopwords removal
data_test['data_text_2'] = data_test['data_text_1'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=True, keep_semantic_punctuation=False, use_stemming=False))
data_test.to_csv('test_all_coloumns.csv')

In [116]:

# Add punctuation handling
data_train['data_text_3'] = data_train['data_text_2'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=True, use_stemming=False))
data_train.to_csv('train_all_coloumns.csv')

In [117]:

# Add punctuation handling
data_test['data_text_3'] = data_test['data_text_2'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=True, use_stemming=False))
data_test.to_csv('test_all_coloumns.csv')

In [118]:

# Incorporate lemmatization
data_train['data_text_4'] = data_train['data_text_3'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=False, combine_negations=True))
data_train.to_csv('train_all_coloumns.csv')

In [119]:

# Incorporate lemmatization
data_test['data_text_4'] = data_test['data_text_3'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=False, combine_negations=True))
data_test.to_csv('test_all_coloumns.csv')

In [17]:

# Incorporate stemming or lemming, depends on which was is more efficent
data_train['data_text_5_stemming'] = data_train['data_text_4'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=False, use_stemming=True, combine_negations=False))
data_train.to_csv('train_all_coloumns.csv')

: 

In [ ]:

# Incorporate stemming or lemming, depends on which was is more efficent
data_test['data_text_5_stemming'] = data_test['data_text_4'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=False, use_stemming=True, combine_negations=False))
data_test.to_csv('train_all_coloumns.csv')

In [ ]:
data_train['data_text_5_lemming'] = data_train['data_text_4'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=False, use_lemming=True, combine_negations=False))
data_train.to_csv('train_all_coloumns.csv')

In [ ]:
data_test['data_text_5_lemming'] = data_test['data_text_4'].apply(lambda x: preprocess_text(x, use_lower=False, remove_stopwords=False, keep_semantic_punctuation=False, use_lemming=True, combine_negations=False))
data_test.to_csv('train_all_coloumns.csv')

In [90]:
data_train = pd.read_csv("./train_all_coloumns.csv", index_col=0)

data_test = pd.read_csv("./test_all_coloumns.csv", index_col=0)
data_train.head(3)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_hate,text_with_punctuation,text_without_negations,...,yes,yesterday,yet,york,you,youi,young,yourselfgo,youtube,zero
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.352046,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
data_test.to_csv('test_all_coloumns.csv')
data_train.to_csv('train_all_coloumns.csv')

In [ ]:

data_test.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_hate,text_no_punctuation,text_with_punctuation,text_without_negations,text_with_negations,data_text_1,data_text_2,data_text_3,data_text_4
id,,,,,,,,,,,,,,,,
0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,0,thank understanding think highly would revert ...,thank understanding think highly would revert ...,thank understand think highly would revert wit...,thank understand think highly would revert wit...,thank you for understanding i think very highl...,thank understanding think highly would revert ...,thank understanding think highly would revert ...,thank understanding think highly would revert ...
000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,0,dear god site horrible,dear god site horrible,dear god site horrible,dear god site horrible,dear god this site is horrible,dear god site horrible,dear god site horrible,dear god site horrible
0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,0,somebody invariably try add religion really me...,somebody invariably try add religion really me...,somebody invariably try add religion really me...,somebody invariably try add religion really me...,somebody will invariably try to add religion r...,somebody invariably try add religion really me...,somebody invariably try add religion really me...,somebody invariably try add religion really me...
0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,0,says right type type institution needed case t...,says right type type institution needed case t...,say right type type institution need case thre...,say right type type institution need case thre...,it says it right there that it is a type the t...,says right type type institution needed case t...,says right type type institution needed case t...,says right type type institution needed case t...
00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,0,adding new product list make sure s relevant a...,adding new product list make sure s relevant a...,add new product list make sure s relevant add ...,add new product list make sure s relevant add ...,before adding a new product to the list make s...,adding new product list make sure relevant add...,adding new product list make sure relevant add...,adding new product list make sure relevant add...


In [121]:
print(data_train.isna().sum())
print(data_test.isna().sum())

comment_text              0
toxic                     0
severe_toxic              0
obscene                   0
threat                    0
insult                    0
identity_hate             0
is_hate                   0
text_no_punctuation       0
text_with_punctuation     0
text_without_negations    0
text_with_negations       0
data_text_1               0
data_text_2               0
data_text_3               0
data_text_4               0
dtype: int64
comment_text              0
toxic                     0
severe_toxic              0
obscene                   0
threat                    0
insult                    0
identity_hate             0
is_hate                   0
text_no_punctuation       0
text_with_punctuation     0
text_without_negations    0
text_with_negations       0
data_text_1               0
data_text_2               0
data_text_3               0
data_text_4               0
dtype: int64


In [122]:
data_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_hate,text_no_punctuation,text_with_punctuation,text_without_negations,text_with_negations,data_text_1,data_text_2,data_text_3,data_text_4
id,,,,,,,,,,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,explanation edits made username hardcore metal...,explanation edits made username hardcore metal...,explanation edits make username hardcore metal...,explanation edits make username hardcore metal...,explanation why the edits made under my userna...,explanation edits made username hardcore metal...,explanation edits made username hardcore metal...,explanation edits made username hardcore metal...
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,daww matches background colour m seemingly stu...,daww matches background colour m seemingly stu...,daww match background colour m seemingly stuck...,daww match background colour m seemingly stuck...,he matches this background colour i seemingly ...,matches background colour seemingly stuck than...,matches background colour seemingly stuck than...,matches background colour seemingly stuck than...
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,hey man m really trying edit war s guy constan...,hey man m really trying edit war s guy constan...,hey man m really try edit war s guy constantly...,hey man m really try edit war s guy constantly...,hey man i really not trying to edit war it jus...,hey man really trying edit war guy constantly ...,hey man really trying edit war guy constantly ...,hey man really trying edit war guy constantly ...
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,ca nt make real suggestions improvement wonder...,ca nt make real suggestions improvement wonder...,ca nt make real suggestion improvement wonder ...,ca nt make real suggestion improvement wonder ...,more i ca make any real suggestions on improve...,ca make real suggestions improvement wondered ...,ca make real suggestions improvement wondered ...,ca make real suggestions improvement wondered ...
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,sir hero chance remember page s,sir hero chance remember page s,sir hero chance remember page s,sir hero chance remember page s,you sir are my hero any chance you remember wh...,sir hero chance remember page,sir hero chance remember page,sir hero chance remember page


In [124]:
# Train and Test set undersample test set
# Define test and train set
def datasetDefinition(columnName):
    X_train = data_train[columnName]
    y_train = data_train["is_hate"]

    X_test = data_test[columnName]
    y_test = data_test["is_hate"]
    return X_train, y_train, X_test, y_test


## Word Embedding



**Notes**: Tokenizing with TF-IDF

In [81]:
tfidf_vectorizer = TfidfVectorizer(max_features=2000)



In [94]:
print(data_test.shape)
data_test.dropna(inplace=True)
print(data_test.shape)


(63978, 16)
(63720, 16)


In [97]:
data_test['text_no_punctuation'].shape

(63720,)

In [127]:

from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import train_test_split


def makeToTensors(X_train, y_train, X_test, y_test):
    # Make the test and train sets to tensors and apply TF-IDF
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

    # Transform testing data
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    # Convert TF-IDF matrices to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_tfidf.toarray(), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    X_train, X_test, y_train, y_test = train_test_split(X_train_tensor, y_train_tensor, test_size=0.8, stratify=y_train, random_state=42)
    # Initialize RandomUnderSampler
    undersampler = RandomUnderSampler(random_state=42)

    # Resample the data
    X_resampled, y_resampled = undersampler.fit_resample(X_test_tensor, y_test_tensor)


    return X_train, y_train, X_resampled, y_resampled

## Model Implementation & Test with Testset

**Note**: Does a CNN makes sense for sentiment analysis? or a simpler model?

**Answers and additional Notes**:
Make a CNN with PyTorch using skorch as wrapper to make it possible to use sklearn.pipeline with the model
This way gridsearch for hyper parameters is possible and tfidfVectorizer can be used for tf-idf
CNN: vector size 300, conv. layer of some size, flatten, relu, end with softmax or something
Example: https://www.kaggle.com/code/raviusz/jigsaw-toxic-comment
example look very good to get basics and then change some of architecture
hyperparameter tuning for each model? only if time permits, alt. tune on best model and use for rest

**Note**: We will use the given test set to compare the different approaches. Make a dataframe with all the results
in accuracy, f1, recall, etc. 

In [128]:

# CNN: The basic model

class CNN(nn.Module):
    def __init__(self, dropout_prob=0.5):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv_output_size = self._get_conv_output_size(2000)
        
        self.fc1 = nn.Linear(self.conv_output_size, 64)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
        
    def _get_conv_output_size(self, input_size):
        x = torch.randn(1, 1, input_size)  # Add channel dimension
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
        return x.view(1, -1).size(1)


batchSize = 25

def trainCNN(X_train, y_train, X_test, y_test,  batch_size=batchSize, epochs=7, learning_rate=0.001):

    # Step 4: Train the model
    train_dataset = TensorDataset(X_train, y_train)  # Assuming X_train_tensor and y_train_tensor are tensors
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    train_loss = []
    train_accuracy = []
    # Move model to GPU
    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print(f'learning rate: {learning_rate}, total number of epochs: {epochs}')
    for epoch in range(int(epochs)):
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        epoch_accuracy = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = total_loss / len(train_dataset)
        epoch_accuracy = correct / total
        train_loss.append(epoch_loss)
        train_accuracy.append(epoch_accuracy)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss}, Accuracy: {epoch_accuracy}')

    # Step 5: Evaluate the model
    # Assuming X_test_tensor and y_test_tensor are tensors
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model.eval()
    total_correct = 0
    total_predicted = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_correct += torch.sum(predicted == labels).item()
            total_predicted += len(predicted)
    # Calculate evaluation metrics
    accuracy = total_correct / total_predicted
    
    print(f'Test Accuracy: {accuracy}')
    return accuracy, model, train_loss, train_accuracy


In [129]:
# for all the columns load them into X_train
# make the tensors, train the cnn and thenn take the outcome and add it to a df
columnName = ['text_no_punctuation', 'text_with_punctuation', 'text_without_negations', 'text_with_negations', 
              'data_text_1', 'data_text_2', 'data_text_3', 'data_text_4']

results = []
all_train_loss = []
all_train_accuracy = []
for name in columnName:
    X_train, y_train, X_test, y_test = datasetDefinition(name)
    X_train_T, y_train_T, X_test_T, y_test_T = makeToTensors(X_train, y_train, X_test, y_test)
    
    # Get the test accuracy using the best model
    accuracy, model, train_loss, train_accuracy = trainCNN(X_train_T, y_train_T, X_test_T, y_test_T, epochs=30, learning_rate=0.0001)
    
    all_train_loss.append(train_loss)
    all_train_accuracy.append(train_accuracy)

    plt.plot(train_loss, label=f'{name} Loss')
    plt.plot(train_accuracy, label=f'{name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.title(f'Loss and Accuracy vs. Epochs for {name}')
    plt.legend()
    plt.show()

    results.append({'Column': name, 'Test_Accuracy': accuracy})

results_df = pd.DataFrame(results)
results_df.to_csv('hyperparameter_tuning_results.csv', index=False)

learning rate: 0.0001, total number of epochs: 30
Epoch [1/30], Loss: 0.31321185253870576, Accuracy: 0.8985711599924798
Epoch [2/30], Loss: 0.23580141776741137, Accuracy: 0.9188443943097073
Epoch [3/30], Loss: 0.21611015361300612, Accuracy: 0.9271792943535753
Epoch [4/30], Loss: 0.20212608106894142, Accuracy: 0.9313780785862004
Epoch [5/30], Loss: 0.19138304286623514, Accuracy: 0.9357021996615905
Epoch [6/30], Loss: 0.18256658843851598, Accuracy: 0.93899229178417
Epoch [7/30], Loss: 0.1744304354340347, Accuracy: 0.941561697060851
Epoch [8/30], Loss: 0.1668829640260127, Accuracy: 0.9439117628626935
Epoch [9/30], Loss: 0.16172726659334535, Accuracy: 0.9466378391928307
Epoch [10/30], Loss: 0.15508793214117286, Accuracy: 0.9483612207808485
Epoch [11/30], Loss: 0.14817559423022933, Accuracy: 0.9508052892147647
Epoch [12/30], Loss: 0.14193019108316718, Accuracy: 0.9518706523782666
Epoch [13/30], Loss: 0.13660416427097186, Accuracy: 0.9538760418625055
Epoch [14/30], Loss: 0.12994292096519153,

In [ ]:
columnName = ['text_lemmatization', 'text_punctuation', 'text_no_punctuation', 'text_negations', 'text_no_negations', 'data_text_1', 'data_text_2',
            'data_text_2', 'data_text_3', 'data_text_4']

for i, name in enumerate(columnName):
    plt.plot(all_train_loss[i], label=f'{name} Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss vs. Epochs for Different Preprocessing Methods')
plt.legend()
plt.show()

for i, name in enumerate(columnName):
    plt.plot(all_train_accuracy[i], label=f'{name} Accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy vs. Epochs for Different Preprocessing Methods')
plt.legend()
plt.show()

IndexError: list index out of range

In [ ]:

columnName = ['text_no_punctuation', 'text_with_punctuation', 'text_without_negations', 
              'text_with_negations', 'data_text_1', 'data_text_2', 'data_text_3', 'data_text_4']

metrics_dict = {'accuracy': [], 'f1-score': [], 'recall': []}
linear_pipeline = make_pipeline(
    TfidfVectorizer(max_features=1000),
    LogisticRegression(solver='sag', max_iter=1000)
)
for name in columnName:
    X_train, y_train, X_test, y_test = datasetDefinition(name)
    linear_pipeline.fit(X_train, y_train)
    y_pred = linear_pipeline.predict(X_test)
    print(name, classification_report(y_test, y_pred))
    metrics_dict['accuracy'].append(classification_report['accuracy_score'])
    metrics_dict['f1-score'].append(classification_report['macro avg']['f1-score'])
    metrics_dict['recall'].append(classification_report['macro avg']['recall'])
    

for metric in metrics_dict.keys():
    plt.figure(figsize=(10, 6))
    plt.bar(columnName, metrics_dict[metric], color='skyblue')
    plt.title(f'{metric.capitalize()} for each Column')
    plt.xlabel('Column Name')
    plt.ylabel(metric.capitalize())
    plt.xticks(rotation=45)
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

KeyError: 'text_no_punctuation'